In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - make_rc_test_ssf.ipynb
# AUTHOR - James Lane
# PROJECT - ges-mass
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Make a fake selection function for mock data that looks like a red clump
sample for testing purposes.
'''

__author__ = "James Lane"

In [ ]:
### Imports

## Basic
import numpy as np
import sys, os, pdb, dill as pickle, copy, warnings

## Matplotlib
from matplotlib import pyplot as plt

## Astropy
from astropy import units as apu

## apomock, mwdust
import apomock, mwdust
from apomock.util.util import join_orbs

## galpy
from galpy import orbit, potential

## Project-specific
sys.path.insert(0,'../../../src/')
from ges_mass import mass as pmass
from ges_mass import densprofiles as pdens
from ges_mass import util as putil

### Scale parameters
ro = 8.275
vo = 220
zo = 0.0208 # Bennett+ 2019

### Notebook setup

%matplotlib inline
plt.style.use('../../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

### Preliminaries

In [ ]:
# Pathing
cdict = putil.load_config_to_dict()
keywords = ['DATA_DIR','APOGEE_DR','APOGEE_RESULTS_VERS','GAIA_DR','NDMOD',
            'DMOD_MIN','DMOD_MAX']
data_dir_base,apogee_dr,apogee_results_vers,gaia_dr,ndmod,dmod_min,dmod_max \
    = putil.parse_config_dict(cdict,keywords)
data_dir = data_dir_base+'gaia_apogee/'
gaia_apogee_dir = 'apogee_'+apogee_dr+'_'+apogee_results_vers+'_gaia_'+gaia_dr+'/'

### Real selection function and effective selection function grid for comparison

In [ ]:
# Filenames
apogee_SF_filename = data_dir+gaia_apogee_dir+'apogee_SF.dat'
apogee_effSF_filename = data_dir+gaia_apogee_dir+'apogee_effSF_grid_inclArea_z0.001_logAge10.0.dat'
apogee_effSF_mask_filename = data_dir+gaia_apogee_dir+'apogee_effSF_grid_mask_z0.001_logAge10.0.npy'

# Selection function
print('APOGEE data release is: '+apogee_dr+', and results version is: '+apogee_results_vers)
if os.path.exists(apogee_SF_filename):
    print('Loading APOGEE selection function from '+apogee_SF_filename)
    with open(apogee_SF_filename, 'rb') as f:
        apogee_SF = pickle.load(f)
else:
    sys.exit('Could not find APOGEE selection function, make it. Exiting...')

# Load the effective selection function grid
if os.path.exists(apogee_effSF_filename):
    print('\nLoading effective selection function from '+apogee_effSF_filename)
    with open(apogee_effSF_filename,'rb') as f:
        apogee_effSF_grid_inclArea = pickle.load(f)
else:
    sys.exit('\nFailed to load effective survey section function grid')
    
# Load the effecive selection function mask
if os.path.exists(apogee_effSF_mask_filename):
    print('\nLoading effective selection function mask from '+apogee_effSF_mask_filename)
    apogee_effSF_mask = np.load(apogee_effSF_mask_filename)
else:
    sys.exit('\nFailed to load effective survey section function grid mask')

In [ ]:
def make_distance_grid(n,dist_min,dist_max):
    dist = np.linspace(dist_min, dist_max, n)
    dmod = 5*np.log10(dist)+10
    return dist,dmod

In [ ]:
# Distance modulus grid
dmods,ds = putil.make_dmod_grid(ndmod,dmod_min,dmod_max)

# Grid of positions in the APOGEE effective selection function grid
Rgrid,phigrid,zgrid = pmass.Rphizgrid(apogee_SF,dmods)

# Apply the effective selection function grid mask
apof_mask = apogee_effSF_grid_inclArea[apogee_effSF_mask]
Rgrid_mask = Rgrid[apogee_effSF_mask]
phigrid_mask = phigrid[apogee_effSF_mask]
zgrid_mask = zgrid[apogee_effSF_mask]

# Include the distance factor in the selection function
apof_mask *= ds**3.*np.log(10)/5.*(dmods[1]-dmods[0])*(np.pi/180.)**2.
#ds_for_delta = 10**(np.append(dmods-(dmods[1]-dmods[0]),dmods[-1]+(dmods[1]-dmods[0]))/5.-2.)
#apof = apof * ds**2. * np.diff(ds_for_delta)

### Now choose an absolute magnitude for the red clump sample & APOGEE magnitude targeting range

Absolute magnitude will be $M_H = -1.5$

APOGEE $H_{min}$ will be the min of the short cohort: $M_H=7$

APOGEE $H_{max}$ will be the max of the long cohorts: $M_H=13.8$

The effective selection function is then simply True (1) if the magnitude is
within this range and False (0) otherwise

In [ ]:
# Hmins
print('Hmins: short, medium, long')
hmins = [apogee_SF._short_hmin,apogee_SF._medium_hmin,apogee_SF._long_hmin]
for i in range(3):
    print(np.nanmin(hmins[i]))

# Hmaxs
print('Hmaxs: short, medium, long')
hmaxs = [apogee_SF._short_hmax,apogee_SF._medium_hmax,apogee_SF._long_hmax]
for i in range(3):
    print(np.nanmax(hmaxs[i]))

In [ ]:
MH = -1.5
Hmin = 7
Hmax = 13.8

def apogee_rc_ssf(H,Hmin=12.2,Hmax=13.8):
    if isinstance(H,(np.ndarray,list)):
        return ((H<Hmax)&(H>Hmin)).astype(float)
    else:
        if (H<Hmax)&(H>Hmin):
            return 1.
        else:
            return 0.
        

In [ ]:
fig = plt.figure(figsize=(10,3))
axs = fig.subplots(nrows=1,ncols=2)

axs[0].plot(dmods,apogee_rc_ssf(dmods+MH))
axs[0].set_xlabel('Distance modulus')
axs[0].set_ylabel('Selection function')

ds_lin,dmod_lin = make_distance_grid(ndmod,0.25,70)
axs[1].plot(ds_lin,apogee_rc_ssf(dmod_lin+MH))
axs[1].set_xlabel('Distance [kpc]')
axs[1].set_ylabel('Selection functions')

fig.show()

### Now create the effective selection function grid
Do it linear in distance modulus and linear in distance

In [ ]:
dmodgrid = np.zeros_like(Rgrid)
for i in range(dmodgrid.shape[0]):
    dmodgrid[i] = dmods
#dmodgrid = dmodgrid[apogee_effSF_mask]
rc_effSF_grid = apogee_rc_ssf(dmodgrid)
rc_effSF_grid *= ds**3.*np.log(10)/5.*(dmods[1]-dmods[0])*(np.pi/180.)**2.
rc_effSF_grid = rc_effSF_grid[apogee_effSF_mask] 

### Now make a mock

In [ ]:
dmap = mwdust.Zero(filter='2MASS H')
alpha = 3.5
rmin = 0.1/ro
rmax = 70./ro
denspot = potential.PowerSphericalPotential(alpha=alpha,ro=ro,vo=vo)
mock = apomock.APOGEEMock(denspot=denspot)
mock.masses = np.ones(int(1e5))
mock.sample_positions(r_min=rmin, r_max=rmax)
mock.apply_selection_function_rc_test(apogee_SF,apogee_rc_ssf,dmap)

In [ ]:
# Properites
orbs = mock.orbs
locid = mock.locid

# Masking
# Cut bulge fields. Within 20 degrees of the galactic center
omask_bulge = ~(((orbs.ll().value > 340.) |\
                (orbs.ll().value < 20.)) &\
               (np.fabs(orbs.bb().value) < 20.)
              )
# Cut fields containing enhancements of globular cluster stars
gc_locid = [2011,4353,5093,5229,5294,5295,5296,5297,5298,5299,5300,5325,5328,
            5329,5438,5528,5529,5744,5801]
omask_gc = ~np.isin(locid,gc_locid)
omask = omask_bulge & omask_gc

orbs = orbs[omask]
print('Number of masked orbs: '+str(len(orbs)))
data_R = orbs.R().value
data_phi = orbs.phi().value
data_z = orbs.z().value

In [ ]:
n_sample = 100
model = pdens.spherical
alpha_range = np.linspace(0.05,5.,num=n_sample)
alpha_correct = 3.5
likelihood = np.zeros_like(alpha_range)

for i in range(len(alpha_range)):
    likelihood[i] = pmass.mloglike([alpha_range[i]], model, rc_effSF_grid,
                                   Rgrid_mask, phigrid_mask, zgrid_mask, 
                                   data_R, data_phi, data_z)

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(alpha_range, likelihood)
ax.axvline(alpha_range[np.argmin(likelihood)], linestyle='dashed', label=r'Min $\mathcal{L}$')
ax.axvline(alpha_correct, linestyle='dashed', color='DarkOrange', label='correct')
ax.legend()
ax.set_xlabel(r'$\alpha$')
ax.set_ylabel(r'$\log \mathcal{L}$')

fig.show()

In [ ]:
alpha = 3.5
rmin = 0.1/ro
rmax = 70./ro
denspot = potential.PowerSphericalPotential(alpha=alpha,ro=ro,vo=vo)
dmap = mwdust.Zero(filter='2MASS H')

ntry = 10

fig = plt.figure()
ax = fig.add_subplot(111)
ax.axvline(alpha, linestyle='dashed', color='DarkOrange', label='correct')
ax.set_xlabel(r'$\alpha$')
ax.set_ylabel(r'$\log \mathcal{L}$')

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for i in range(ntry):

        mock = apomock.APOGEEMock(denspot=denspot)
        mock.masses = np.ones(int(1e5))
        mock.sample_positions(r_min=rmin, r_max=rmax)
        mock.apply_selection_function_rc_test(apogee_SF,apogee_rc_ssf,dmap)

        # Properites
        orbs = mock.orbs
        locid = mock.locid

        # Masking
        # Cut bulge fields. Within 20 degrees of the galactic center
        omask_bulge = ~(((orbs.ll().value > 340.) |\
                        (orbs.ll().value < 20.)) &\
                       (np.fabs(orbs.bb().value) < 20.)
                      )
        # Cut fields containing enhancements of globular cluster stars
        gc_locid = [2011,4353,5093,5229,5294,5295,5296,5297,5298,5299,5300,5325,5328,
                    5329,5438,5528,5529,5744,5801]
        omask_gc = ~np.isin(locid,gc_locid)
        omask = omask_bulge & omask_gc

        orbs = orbs[omask]
        print('Number of masked orbs: '+str(len(orbs)))
        data_R = orbs.R().value
        data_phi = orbs.phi().value
        data_z = orbs.z().value

        n_sample = 100
        model = pdens.spherical
        alpha_range = np.linspace(0.05,5.,num=n_sample)
        alpha_correct = 3.5
        likelihood = np.zeros_like(alpha_range)

        for j in range(len(alpha_range)):
            likelihood[j] = pmass.mloglike([alpha_range[j]], model, rc_effSF_grid,
                                           Rgrid_mask, phigrid_mask, zgrid_mask, 
                                           data_R, data_phi, data_z)

        ax.plot(alpha_range, likelihood)
        if i == ntry-1:
            ax.axvline(alpha_range[np.argmin(likelihood)], linestyle='dashed', 
                       label=r'Min $\mathcal{L}$', alpha=0.5)
            ax.legend()
        else:
            ax.axvline(alpha_range[np.argmin(likelihood)], linestyle='dashed',
                       alpha=0.5)
        ax.set_xlabel(r'$\alpha$')
        ax.set_ylabel(r'$\log \mathcal{L}$')

In [ ]:
alpha = 3.5
rmin = 1.0/ro
rmax = 70./ro
denspot = potential.PowerSphericalPotential(alpha=alpha,ro=ro,vo=vo)
dmap = mwdust.Zero(filter='2MASS H')

ntry = 10

fig = plt.figure()
ax = fig.add_subplot(111)
ax.axvline(alpha, linestyle='dashed', color='DarkOrange', label='correct')
ax.set_xlabel(r'$\alpha$')
ax.set_ylabel(r'$\log \mathcal{L}$')

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for i in range(ntry):

        mock = apomock.APOGEEMock(denspot=denspot)
        mock.masses = np.ones(int(1e5))
        mock.sample_positions(r_min=rmin, r_max=rmax)
        mock.apply_selection_function_rc_test(apogee_SF,apogee_rc_ssf,dmap)

        # Properites
        orbs = mock.orbs
        locid = mock.locid

        # Masking
        # Cut bulge fields. Within 20 degrees of the galactic center
        omask_bulge = ~(((orbs.ll().value > 340.) |\
                        (orbs.ll().value < 20.)) &\
                       (np.fabs(orbs.bb().value) < 20.)
                      )
        # Cut fields containing enhancements of globular cluster stars
        gc_locid = [2011,4353,5093,5229,5294,5295,5296,5297,5298,5299,5300,5325,5328,
                    5329,5438,5528,5529,5744,5801]
        omask_gc = ~np.isin(locid,gc_locid)
        omask = omask_bulge & omask_gc

        orbs = orbs[omask]
        print('Number of masked orbs: '+str(len(orbs)))
        data_R = orbs.R().value
        data_phi = orbs.phi().value
        data_z = orbs.z().value

        n_sample = 100
        model = pdens.spherical
        alpha_range = np.linspace(0.05,5.,num=n_sample)
        alpha_correct = 3.5
        likelihood = np.zeros_like(alpha_range)

        for j in range(len(alpha_range)):
            likelihood[j] = pmass.mloglike([alpha_range[j]], model, rc_effSF_grid,
                                           Rgrid_mask, phigrid_mask, zgrid_mask, 
                                           data_R, data_phi, data_z)

        ax.plot(alpha_range, likelihood)
        if i == ntry-1:
            ax.axvline(alpha_range[np.argmin(likelihood)], linestyle='dashed', 
                       label=r'Min $\mathcal{L}$', alpha=0.5)
            ax.legend()
        else:
            ax.axvline(alpha_range[np.argmin(likelihood)], linestyle='dashed',
                       alpha=0.5)
        ax.set_xlabel(r'$\alpha$')
        ax.set_ylabel(r'$\log \mathcal{L}$')

In [ ]:
alpha = 2.5
rmin = 0.1/ro
rmax = 70./ro
denspot = potential.PowerSphericalPotential(alpha=alpha,ro=ro,vo=vo)
dmap = mwdust.Zero(filter='2MASS H')

ntry = 10

fig = plt.figure()
ax = fig.add_subplot(111)
ax.axvline(alpha, linestyle='dashed', color='DarkOrange', label='correct')
ax.set_xlabel(r'$\alpha$')
ax.set_ylabel(r'$\log \mathcal{L}$')

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for i in range(ntry):

        mock = apomock.APOGEEMock(denspot=denspot)
        mock.masses = np.ones(int(1e5))
        mock.sample_positions(r_min=rmin, r_max=rmax)
        mock.apply_selection_function_rc_test(apogee_SF,apogee_rc_ssf,dmap)

        # Properites
        orbs = mock.orbs
        locid = mock.locid

        # Masking
        # Cut bulge fields. Within 20 degrees of the galactic center
        omask_bulge = ~(((orbs.ll().value > 340.) |\
                        (orbs.ll().value < 20.)) &\
                       (np.fabs(orbs.bb().value) < 20.)
                      )
        # Cut fields containing enhancements of globular cluster stars
        gc_locid = [2011,4353,5093,5229,5294,5295,5296,5297,5298,5299,5300,5325,5328,
                    5329,5438,5528,5529,5744,5801]
        omask_gc = ~np.isin(locid,gc_locid)
        omask = omask_bulge & omask_gc

        orbs = orbs[omask]
        print('Number of masked orbs: '+str(len(orbs)))
        data_R = orbs.R().value
        data_phi = orbs.phi().value
        data_z = orbs.z().value

        n_sample = 100
        model = pdens.spherical
        alpha_range = np.linspace(0.05,5.,num=n_sample)
        alpha_correct = 3.5
        likelihood = np.zeros_like(alpha_range)

        for j in range(len(alpha_range)):
            likelihood[j] = pmass.mloglike([alpha_range[j]], model, rc_effSF_grid,
                                           Rgrid_mask, phigrid_mask, zgrid_mask, 
                                           data_R, data_phi, data_z)

        ax.plot(alpha_range, likelihood)
        if i == ntry-1:
            ax.axvline(alpha_range[np.argmin(likelihood)], linestyle='dashed', 
                       label=r'Min $\mathcal{L}$', alpha=0.5)
            ax.legend()
        else:
            ax.axvline(alpha_range[np.argmin(likelihood)], linestyle='dashed',
                       alpha=0.5)
        ax.set_xlabel(r'$\alpha$')
        ax.set_ylabel(r'$\log \mathcal{L}$')

In [ ]:
alpha = 2.5
rmin = 1.0/ro
rmax = 70./ro
denspot = potential.PowerSphericalPotential(alpha=alpha,ro=ro,vo=vo)
dmap = mwdust.Zero(filter='2MASS H')

ntry = 10

fig = plt.figure()
ax = fig.add_subplot(111)
ax.axvline(alpha, linestyle='dashed', color='DarkOrange', label='correct')
ax.set_xlabel(r'$\alpha$')
ax.set_ylabel(r'$\log \mathcal{L}$')

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for i in range(ntry):

        mock = apomock.APOGEEMock(denspot=denspot)
        mock.masses = np.ones(int(1e5))
        mock.sample_positions(r_min=rmin, r_max=rmax)
        mock.apply_selection_function_rc_test(apogee_SF,apogee_rc_ssf,dmap)

        # Properites
        orbs = mock.orbs
        locid = mock.locid

        # Masking
        # Cut bulge fields. Within 20 degrees of the galactic center
        omask_bulge = ~(((orbs.ll().value > 340.) |\
                        (orbs.ll().value < 20.)) &\
                       (np.fabs(orbs.bb().value) < 20.)
                      )
        # Cut fields containing enhancements of globular cluster stars
        gc_locid = [2011,4353,5093,5229,5294,5295,5296,5297,5298,5299,5300,5325,5328,
                    5329,5438,5528,5529,5744,5801]
        omask_gc = ~np.isin(locid,gc_locid)
        omask = omask_bulge & omask_gc

        orbs = orbs[omask]
        print('Number of masked orbs: '+str(len(orbs)))
        data_R = orbs.R().value
        data_phi = orbs.phi().value
        data_z = orbs.z().value

        n_sample = 100
        model = pdens.spherical
        alpha_range = np.linspace(0.05,5.,num=n_sample)
        alpha_correct = 3.5
        likelihood = np.zeros_like(alpha_range)

        for j in range(len(alpha_range)):
            likelihood[j] = pmass.mloglike([alpha_range[j]], model, rc_effSF_grid,
                                           Rgrid_mask, phigrid_mask, zgrid_mask, 
                                           data_R, data_phi, data_z)

        ax.plot(alpha_range, likelihood)
        if i == ntry-1:
            ax.axvline(alpha_range[np.argmin(likelihood)], linestyle='dashed', 
                       label=r'Min $\mathcal{L}$', alpha=0.5)
            ax.legend()
        else:
            ax.axvline(alpha_range[np.argmin(likelihood)], linestyle='dashed',
                       alpha=0.5)
        ax.set_xlabel(r'$\alpha$')
        ax.set_ylabel(r'$\log \mathcal{L}$')